In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import keyboard

In [2]:
import run_parallel

In [3]:
# Running in parallel
run_parallel.main()

ep
THIS IS ORIGINAL FOLDER PATH /Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Code/SensUs_2021_Code
/Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Code/SensUs_2021_Code/focus
['test.tiff', 'test2.tiff', 'test2 còpia.tiff', '.gitkeep', 'test2 còpia 2.tiff']
ROI PATH /Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Code/SensUs_2021_Code/focus/test2 còpia 2.tiff
Select the ROI. Press right button if you want to delete. The last 2 ROIs will be used as background. Press 'q' when you have finished. 
ROIs:
 [[2876 1596  480]
 [ 608 2020  480]
 [2120 3080  480]
 [3932 2768  480]]
OBSERVED FOLDER /Users/janet/EnginyeriaBiomèdica/MSc LIFE SCIENCES ENGINEERING/SensUs/Code/SensUs_2021_Code/images/20210828
Press "s" or ctrl+c to exit and save the results


<Figure size 504x360 with 0 Axes>

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Applications/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda3/lib/python3.7/site-packages/keyboard/__init__.py", line 294, in listen
    _os_keyboard.listen(self.direct_callback)
  File "/Applications/anaconda3/lib/python3.7/site-packages/keyboard/_darwinkeyboard.py", line 430, in listen
    raise OSError("Error 13 - Must be run as administrator")
OSError: Error 13 - Must be run as administrator



<Figure size 504x360 with 0 Axes>

<Figure size 504x360 with 0 Axes>

Last results list [[]]
y []
x []


TypeError: expected non-empty vector for x

In [ ]:
# Running pump TODO

In [ ]:
# Loading results
result_df = pd.read_csv('./images/20210828/result.csv')   # TODO CHANGE NAME
result_df['Signal'].plot()

In [ ]:
print(keyboard.is_pressed('ctrl'))

In [ ]:
while True:  # making a loop
    try:  # used try so that if user pressed other than the given key error will not be shown
        if keyboard.is_pressed('q'):  # if key 'q' is pressed 
            print('You Pressed A Key!')
            break  # finishing the loop
    except:
        break  # if user pressed a key other than the given key the loop will break

In [ ]:
# todo: median filter